In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
root_path = '/content/gdrive/My Drive/Colab Notebooks/The Movie Dataset Extracted'
dataset = pd.read_csv(root_path+"/ratings_small.csv", sep = ',')

In [ ]:
dataset.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
len(dataset.userId.unique())

671

In [ ]:
len(dataset.movieId.unique())

9066

In [ ]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(dataset,test_size= 0.2) 

In [ ]:
train.head(32)

,userId,movieId,rating,timestamp
60119,434,595,5.0,886374784
26573,195,2617,3.0,975418661
65108,463,2735,3.0,1051712464
90794,603,1017,4.0,868355519
68342,475,5952,5.0,1446908199
43148,308,407,1.0,854376743
21571,150,1617,3.0,1114306474
88186,587,265,3.5,1111102715
23296,165,1259,3.5,1111479987
43821,311,1936,3.5,1077051541


In [ ]:
test.head(15)

,userId,movieId,rating,timestamp
87093,580,4963,1.5,1167156856
5700,30,3316,2.0,994439603
5053,30,14,4.0,945276564
5081,30,203,3.0,994458432
62330,452,4025,1.5,1151811810
61774,452,1480,3.0,1077114310
622,8,1674,4.0,1154389491
75858,527,55267,5.0,1281232639
86387,577,5502,4.0,1111551392
22725,159,1580,3.5,1183519595


In [ ]:
len(test), len(train)

(20001, 80003)

In [ ]:
len(dataset.userId.unique()),len(dataset.movieId.unique())

(671, 9066)

In [ ]:
dataset

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [ ]:
import tensorflow as tf
print (tf.__version__)

1.15.0


In [ ]:
max_movieId = dataset["movieId"].max()
print(max_movieId)
max_userId = dataset["userId"].max()

163949


In [ ]:
max_movieId = dataset["movieId"].max()
print(max_movieId)
max_userId = dataset["userId"].max()
print(max_userId)

163949
671


In [ ]:
import keras
from keras.optimizers import Adam
from keras.layers import Dense, Dropout
n_latent_factors_users = 10
n_latent_factors_movie =10
n_latent_factors_mf = 10
max_movieId = dataset["movieId"].max()
max_userId = dataset["userId"].max()
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())#dont use this in embedding layer because the vocabulary size will be the number of unique
#movies and we wouldn't be able to test movies which have a bigger movieId than total no of unique movies in the dataset

#defining the shape of the input tensor
movie_input = keras.layers.Input(shape= [1], name ='Item')

movie_embedding_mlp = keras.layers.Embedding(max_movieId + 1, n_latent_factors_movie, name='Movie-Embeddding-MLP') (movie_input)
movie_vec_mlp = keras.layers.Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.5)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(max_movieId + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name= 'FlattenMovie-MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.5)(movie_vec_mf)



user_input = keras.layers.Input( shape= [1] , name ='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUser-MLP')(keras.layers.Embedding(n_users +1, n_latent_factors_users, name='User_Embedding_MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.5)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users +1, n_latent_factors_mf , name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.5)(user_vec_mf)

#merging the layers
concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp],name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)


dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.dot([movie_vec_mf, user_vec_mf], axes= 1 , name='Dot')


pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = keras.layers.concatenate([pred_mf, pred_mlp], name='Concat-MF-MLP')
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)


result = keras.layers.Dense(1,name='Prediction')(deep_combine)


model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_squared_error')

In [ ]:
history = model.fit([train.userId, train.movieId], train.rating, epochs=20, verbose=1, validation_split=0.1)




Train on 72002 samples, validate on 8001 samples
Epoch 1/20





72002/72002 [==============================] - 38s 534us/step - loss: 1.2040 - val_loss: 1.0118
Epoch 2/20
72002/72002 [==============================] - 28s 388us/step - loss: 0.9756 - val_loss: 0.9115
Epoch 3/20
72002/72002 [==============================] - 28s 387us/step - loss: 0.8989 - val_loss: 0.8415
Epoch 4/20
72002/72002 [==============================] - 28s 382us/step - loss: 0.8499 - val_loss: 0.8341
Epoch 5/20
72002/72002 [==============================] - 28s 388us/step - loss: 0.8147 - val_loss: 0.8132
Epoch 6/20
72002/72002 [==============================] - 27s 379us/step - loss: 0.7895 - val_loss: 0.8073
Epoch 7/20
72002/72002 [==============================] - 27s 381us/step - loss: 0.7589 - val_loss: 0.8224
Epoch 8/20
72002/72002 [==============================] - 27s 379us/step - loss: 0.7481 - val_loss: 0.8047
Epoch 9/20
72002/72002 [==============================] - 28s 389us/step - loss: 0.7325

In [ ]:
from sklearn.metrics import mean_absolute_error
y_true = test.rating
y_hat_2 = np.round(model.predict([test.userId, test.movieId]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.userId, test.movieId])))

0.6700914954252287
0.6771892690699098


In [ ]:
from sklearn.metrics import mean_squared_error
y_true = test.rating
y_hat_2 = np.round(model.predict([test.userId, test.movieId]),0)
#print(mean_squared_error(y_true, y_hat_2))

print(mean_squared_error(y_true, model.predict([test.userId, test.movieId])))

0.7842729922171395


In [ ]:
model.save('/content/gdrive/My Drive/Colab Notebooks/collab_trained_smaller.h5')

In [ ]:
prediction= model.predict([test.userId,test.movieId],batch_size =10, verbose=1)

20001/20001 [==============================] - 4s 222us/step


In [ ]:
from keras.models import load_model
new_model=load_model('/content/gdrive/My Drive/Colab Notebooks/collab_trained_smaller.h5')


In [ ]:
for i in prediction:
  print(i)

Streaming output truncated to the last 5000 lines.
[4.240862]
[4.2718315]
[4.0193815]
[4.4087067]
[3.3049703]
[2.3969872]
[3.610729]
[3.7617161]
[4.374179]
[3.1573973]
[4.1644]
[3.930697]
[3.8543754]
[3.4640515]
[4.028056]
[4.4074516]
[2.7828314]
[2.6845546]
[4.0461874]
[3.5953054]
[4.476634]
[3.4972906]
[3.0317907]
[3.246858]
[4.302313]
[3.4064872]
[3.3911154]
[3.1534953]
[4.5096045]
[4.357271]
[4.138113]
[4.4468346]
[3.820061]
[3.7219024]
[4.139898]
[3.199787]
[4.389285]
[3.8021517]
[3.9044867]
[2.9002964]
[4.063635]
[3.5660858]
[3.5218115]
[3.307904]
[4.230874]
[3.4102547]
[2.6845546]
[3.3100333]
[3.770897]
[3.4783359]
[3.3115702]
[3.8708522]
[3.9147053]
[4.3435216]
[2.794929]
[3.308053]
[3.350846]
[3.5230594]
[3.4807706]
[2.6745682]
[3.652842]
[3.7115161]
[4.3104267]
[3.8866496]
[3.9342997]
[2.8351142]
[2.15381]
[2.4275029]
[3.3018172]
[2.503114]
[3.4067082]
[4.046592]
[3.623564]
[3.9946396]
[4.2476974]
[2.8564668]
[4.0792885]
[3.6413374]
[3.0907393]
[3.1652536]
[4.182]
[4.901124]


In [ ]:
new_model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embeddding-MLP (Embedding (None, 1, 100)       16395000    Item[0][0]                       
__________________________________________________________________________________________________
User_Embedding_MLP (Embedding)  (None, 1, 100)       67200       User[0][0]                       
____________________________________________________________________________________________